# Net

In [1]:
import torch
from torchvision.models import AlexNet
from torchviz import make_dot
import torch.nn as nn
CHANNELS = [20, 50, 500]
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata

In [2]:
input=torch.rand(1,12,60,60)

class Net(nn.Module):
    def __init__(self, input_channels):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, CHANNELS[0], 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(CHANNELS[0], CHANNELS[1], 5)
        self.fc1 = nn.Linear(CHANNELS[1] * 12 * 12, CHANNELS[2])
        self.fc2 = nn.Linear(CHANNELS[2], 2)

    def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv1(x))
        print(x.shape)
        x=self.pool(x)
        print(x.shape)
#         x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv2(x))
        print(x.shape)
        x=self.pool(x)
        print(x.shape)
        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        print(x.shape)
        x = F.relu(self.fc1(x))
        print(x.shape)
        x = self.fc2(x)
        print(x.shape)
        return x
model=Net(12)
y=model(input)

torch.Size([1, 20, 56, 56])
torch.Size([1, 20, 28, 28])
torch.Size([1, 50, 24, 24])
torch.Size([1, 50, 12, 12])
torch.Size([1, 7200])
torch.Size([1, 500])
torch.Size([1, 2])


In [3]:
g=make_dot(y)

In [4]:
g.render('espnet_model', view=False) 

'espnet_model.pdf'

In [5]:
params = list(model.parameters())
k = 0
for i in params:
        l = 1
        print("该层的结构：" + str(list(i.size())))
        for j in i.size():
                l *= j
        print("该层参数和：" + str(l))
        k = k + l
print("总参数数量和：" + str(k))


该层的结构：[20, 12, 5, 5]
该层参数和：6000
该层的结构：[20]
该层参数和：20
该层的结构：[50, 20, 5, 5]
该层参数和：25000
该层的结构：[50]
该层参数和：50
该层的结构：[500, 7200]
该层参数和：3600000
该层的结构：[500]
该层参数和：500
该层的结构：[2, 500]
该层参数和：1000
该层的结构：[2]
该层参数和：2
总参数数量和：3632572


# 实验1

In [26]:
input=torch.rand(64,12,60,60)
class one_one_Net(nn.Module):
    def __init__(self, input_channels):
        super(one_one_Net, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, CHANNELS[0], 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(CHANNELS[0], CHANNELS[1], 5)

        #         self.fc1 = nn.Linear(CHANNELS[1] * 12 * 12, CHANNELS[2])
        self.softconv1 = nn.Conv2d(in_channels=50, out_channels=25, kernel_size=1, stride=1, padding=0)
        self.softconv2 = nn.Conv2d(in_channels=25, out_channels=50, kernel_size=1, stride=1, padding=0)
        self.softconv3 = nn.Conv2d(in_channels=50, out_channels=2, kernel_size=1, stride=1, padding=0)

    #         self.fc2 =nn.Linear(1,CHANNELS[2])
    #         self.fc3 = nn.Linear(CHANNELS[2], 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        print(x.shape)
        #         x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])

        #         x = F.relu(self.fc1(x))
        x = self.softconv1(x)
        x = self.softconv2(x)
        x = self.softconv3(x)
        print(x.shape)
        return x


model=one_one_Net(12)
y=model(input)
g=make_dot(y)
params = list(model.parameters())
k = 0
for i in params:
        l = 1
        print("该层的结构：" + str(list(i.size())))
        for j in i.size():
                l *= j
        print("该层参数和：" + str(l))
        k = k + l
print("总参数数量和：" + str(k))


torch.Size([64, 50, 12, 12])
torch.Size([64, 2, 12, 12])
该层的结构：[20, 12, 5, 5]
该层参数和：6000
该层的结构：[20]
该层参数和：20
该层的结构：[50, 20, 5, 5]
该层参数和：25000
该层的结构：[50]
该层参数和：50
该层的结构：[25, 50, 1, 1]
该层参数和：1250
该层的结构：[25]
该层参数和：25
该层的结构：[50, 25, 1, 1]
该层参数和：1250
该层的结构：[50]
该层参数和：50
该层的结构：[2, 50, 1, 1]
该层参数和：100
该层的结构：[2]
该层参数和：2
总参数数量和：33747


# 实验2    nin  

In [1]:
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..") 
# import d2lzh_pytorch as d2l


def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk

import torch.nn.functional as F
class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])
    
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)


net = nn.Sequential(
    nin_block(12, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=2, stride=1), 
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 2, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(),
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    FlattenLayer())
# X = torch.rand(64,12,60,60)
X = torch.rand(64, 12, 60, 60)
for name, blk in net.named_children():
    
    X = blk(X)
    print(name, 'output shape: ', X.shape)

0 output shape:  torch.Size([64, 96, 13, 13])
1 output shape:  torch.Size([64, 96, 12, 12])
2 output shape:  torch.Size([64, 256, 12, 12])
3 output shape:  torch.Size([64, 256, 11, 11])
4 output shape:  torch.Size([64, 384, 11, 11])
5 output shape:  torch.Size([64, 384, 10, 10])
6 output shape:  torch.Size([64, 384, 10, 10])
7 output shape:  torch.Size([64, 2, 10, 10])
8 output shape:  torch.Size([64, 2, 1, 1])
9 output shape:  torch.Size([64, 2])


# nin网络  加入bn层

In [1]:
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..") 
# import d2lzh_pytorch as d2l
def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y


def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        BatchNorm(out_channels, num_dims=4),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk

import torch.nn.functional as F
class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])
    
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)


net = nn.Sequential(
    nin_block(12, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=2, stride=1), 
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 2, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(),
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    FlattenLayer())
# X = torch.rand(64,12,60,60)
X = torch.rand(64, 12, 60, 60)
for name, blk in net.named_children():
    
    X = blk(X)
    print(name, 'output shape: ', X.shape)
print(200*'-')
model= net

params = list(model.parameters())
k = 0
for i in params:
        l = 1
        print("该层的结构：" + str(list(i.size())))
        for j in i.size():
                l *= j
        print("该层参数和：" + str(l))
        k = k + l
print("总参数数量和：" + str(k))

0 output shape:  torch.Size([64, 96, 13, 13])
1 output shape:  torch.Size([64, 96, 12, 12])
2 output shape:  torch.Size([64, 256, 12, 12])
3 output shape:  torch.Size([64, 256, 11, 11])
4 output shape:  torch.Size([64, 384, 11, 11])
5 output shape:  torch.Size([64, 384, 10, 10])
6 output shape:  torch.Size([64, 384, 10, 10])
7 output shape:  torch.Size([64, 2, 10, 10])
8 output shape:  torch.Size([64, 2, 1, 1])
9 output shape:  torch.Size([64, 2])
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
该层的结构：[96, 12, 11, 11]
该层参数和：139392
该层的结构：[96]
该层参数和：96
该层的结构：[1, 96, 1, 1]
该层参数和：96
该层的结构：[1, 96, 1, 1]
该层参数和：96
该层的结构：[96, 96, 1, 1]
该层参数和：9216
该层的结构：[96]
该层参数和：96
该层的结构：[96, 96, 1, 1]
该层参数和：9216
该层的结构：[96]
该层参数和：96
该层的结构：[256, 96, 5, 5]
该层参数和：614400
该层的结构：[256]
该层参数和：256
该层的结构：[1, 256, 1, 1]
该层参数和：256
该层的结构：[1, 256, 1, 1]
该层参数和：256
该层的结构：[256

# con1 conv2  nin_block

In [1]:
import torch
from torchvision.models import AlexNet
from torchviz import make_dot
import torch.nn as nn
CHANNELS = [20, 50, 500]
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata

input=torch.rand(64,12,60,60)

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        BatchNorm(out_channels, num_dims=4),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk


import torch.nn.functional as F


class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])


class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()

    def forward(self, x):  # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)


net = nn.Sequential(
    #     nin_block(12, 96, kernel_size=11, stride=4, padding=0),
    #     nn.MaxPool2d(kernel_size=2, stride=1),
    #     nin_block(96, 256, kernel_size=5, stride=1, padding=2),

    #     nn.MaxPool2d(kernel_size=2, stride=1),
    nn.Conv2d(12, 20, 5),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(20, 50, 5),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nin_block(50, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=2, stride=1),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 2, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(),
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    FlattenLayer())
    
    
    


# class fcl2one_Net(nn.Module):
#     def __init__(self, input_channels):
#         super(fcl2one_Net, self).__init__()
#         self.conv1 = nn.Conv2d(input_channels, CHANNELS[0], 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(CHANNELS[0], CHANNELS[1], 5)
        
#         nin_block(256, 384, kernel_size=3, stride=1, padding=1)
# #         self.fc1 = nn.Linear(CHANNELS[1] * 12 * 12, CHANNELS[2])
# #         self.fc2 = nn.Linear(CHANNELS[2], 2)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         print(x.shape)
#         x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        
#         x = F.relu(self.fc1(x))
        
#         x = self.fc2(x)
#         print(x.shape)
#         return x



# model=one_one_Net(12)

# y=model(input)
model=net
y=net(input)
g=make_dot(y)
params = list(model.parameters())
k = 0
for i in params:
        l = 1
        print("该层的结构：" + str(list(i.size())))
        for j in i.size():
                l *= j
        print("该层参数和：" + str(l))
        k = k + l
print("总参数数量和：" + str(k))




该层的结构：[20, 12, 5, 5]
该层参数和：6000
该层的结构：[20]
该层参数和：20
该层的结构：[50, 20, 5, 5]
该层参数和：25000
该层的结构：[50]
该层参数和：50
该层的结构：[384, 50, 3, 3]
该层参数和：172800
该层的结构：[384]
该层参数和：384
该层的结构：[1, 384, 1, 1]
该层参数和：384
该层的结构：[1, 384, 1, 1]
该层参数和：384
该层的结构：[384, 384, 1, 1]
该层参数和：147456
该层的结构：[384]
该层参数和：384
该层的结构：[384, 384, 1, 1]
该层参数和：147456
该层的结构：[384]
该层参数和：384
该层的结构：[2, 384, 3, 3]
该层参数和：6912
该层的结构：[2]
该层参数和：2
该层的结构：[1, 2, 1, 1]
该层参数和：2
该层的结构：[1, 2, 1, 1]
该层参数和：2
该层的结构：[2, 2, 1, 1]
该层参数和：4
该层的结构：[2]
该层参数和：2
该层的结构：[2, 2, 1, 1]
该层参数和：4
该层的结构：[2]
该层参数和：2
总参数数量和：507632


In [2]:
# import torch
# import tensorwatch as tw

 
# # # 其实就两句话
# model=net
# tw.draw_model(model, [64, 12, 60, 60])

RuntimeError: Failed to export an ONNX attribute 'onnx::Gather', since it's not constant, please try to make things (e.g., kernel size) static if possible

# one_conv_one_conv    三个（卷积+batchnormalization+1*1卷积）+fl层（用他）

In [10]:
import torch
from torchvision.models import AlexNet
from torchviz import make_dot
import torch.nn as nn
CHANNELS = [20, 50, 500]
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata

input=torch.rand(64,12,60,60)

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

class one_fl_Net_con_nomal(nn.Module):
    def __init__(self, input_channels):
        super(one_fl_Net_con_nomal, self).__init__()

        self.a_1 = nn.Conv2d(12, 96, kernel_size=11, stride=4, padding=0)
        self.batch_1 = BatchNorm(96, num_dims=4)
        self.a_one = nn.Conv2d(96, 96, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.b_1 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.batch_2 = BatchNorm(256, num_dims=4)
        self.b_one = nn.Conv2d(256, 256, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.c_1 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.batch_3 = BatchNorm(384, num_dims=4)
        self.c_one = nn.Conv2d(384, 384, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.consoft = nn.Conv2d(384, 50, kernel_size=1, stride=1, padding=1)
        self.fc1 = nn.Linear(50 * 12 * 12, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.batch_1(self.a_1(x)))
        x = F.relu(self.a_one(F.relu(self.a_one(x))))
        x = self.pool(x)

        x = F.relu(self.batch_2(self.b_1(x)))
        x = F.relu(self.b_one(F.relu(self.b_one(x))))
        x = self.pool(x)

        x = F.relu(self.batch_3(self.c_1(x)))
        x = F.relu(self.c_one(F.relu(self.c_one(x))))
        x = self.pool(x)

        x = self.consoft(x)

        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        print('aaaaa'+x.shape)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model=one_one_Net(12)
y=model(input)
g=make_dot(y)
params = list(model.parameters())
k = 0
for i in params:
        l = 1
        print("该层的结构：" + str(list(i.size())))
        for j in i.size():
                l *= j
        print("该层参数和：" + str(l))
        k = k + l
print("总参数数量和：" + str(k))

该层的结构：[96, 12, 11, 11]
该层参数和：139392
该层的结构：[96]
该层参数和：96
该层的结构：[96, 96, 1, 1]
该层参数和：9216
该层的结构：[96]
该层参数和：96
该层的结构：[256, 96, 5, 5]
该层参数和：614400
该层的结构：[256]
该层参数和：256
该层的结构：[256, 256, 1, 1]
该层参数和：65536
该层的结构：[256]
该层参数和：256
该层的结构：[384, 256, 3, 3]
该层参数和：884736
该层的结构：[384]
该层参数和：384
该层的结构：[384, 384, 1, 1]
该层参数和：147456
该层的结构：[384]
该层参数和：384
该层的结构：[50, 384, 1, 1]
该层参数和：19200
该层的结构：[50]
该层参数和：50
该层的结构：[50, 7200]
该层参数和：360000
该层的结构：[50]
该层参数和：50
该层的结构：[2, 50]
该层参数和：100
该层的结构：[2]
该层参数和：2
总参数数量和：2241610


# one_conv_one_conv    三个（卷积+BN+1*1卷积）+fl层+BN

In [1]:
import torch
from torchvision.models import AlexNet
from torchviz import make_dot
import torch.nn as nn
CHANNELS = [20, 50, 500]
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata

input=torch.rand(64,12,60,60)

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

class one_fl_Net_con_bn_fl_bn(nn.Module):
    def __init__(self, input_channels):
        super(one_fl_Net_con_bn_fl_bn, self).__init__()

        self.a_1 = nn.Conv2d(12, 96, kernel_size=11, stride=4, padding=0)
        self.batch_1 = BatchNorm(96, num_dims=4)
        self.a_one = nn.Conv2d(96, 96, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.b_1 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.batch_2 = BatchNorm(256, num_dims=4)
        self.b_one = nn.Conv2d(256, 256, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.c_1 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.batch_3 = BatchNorm(384, num_dims=4)
        self.c_one = nn.Conv2d(384, 384, kernel_size=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        self.consoft = nn.Conv2d(384, 50, kernel_size=1, stride=1, padding=1)
        self.fc1 = nn.Linear(50 * 12 * 12, 50)
        self.batch_fl = BatchNorm(50, num_dims=2)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.batch_1(self.a_1(x)))
        x = F.relu(self.a_one(F.relu(self.a_one(x))))
        x = self.pool(x)
        print(x.shape)
        x = F.relu(self.batch_2(self.b_1(x)))
        x = F.relu(self.b_one(F.relu(self.b_one(x))))
        x = self.pool(x)
        print(x.shape)
        x = F.relu(self.batch_3(self.c_1(x)))
        x = F.relu(self.c_one(F.relu(self.c_one(x))))
        x = self.pool(x)
        print(x.shape)
        x = self.consoft(x)
        print(x.shape)
        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        x = F.relu(self.batch_fl(self.fc1(x)))
        x = self.fc2(x)
        return x

model=one_fl_Net_con_bn_fl_bn(12)
y=model(input)
g=make_dot(y)
params = list(model.parameters())
k = 0
for i in params:
        l = 1
        print("该层的结构：" + str(list(i.size())))
        for j in i.size():
                l *= j
        print("该层参数和：" + str(l))
        k = k + l
print("总参数数量和：" + str(k))

torch.Size([64, 96, 12, 12])
torch.Size([64, 256, 11, 11])
torch.Size([64, 384, 10, 10])
torch.Size([64, 50, 12, 12])
该层的结构：[96, 12, 11, 11]
该层参数和：139392
该层的结构：[96]
该层参数和：96
该层的结构：[1, 96, 1, 1]
该层参数和：96
该层的结构：[1, 96, 1, 1]
该层参数和：96
该层的结构：[96, 96, 1, 1]
该层参数和：9216
该层的结构：[96]
该层参数和：96
该层的结构：[256, 96, 5, 5]
该层参数和：614400
该层的结构：[256]
该层参数和：256
该层的结构：[1, 256, 1, 1]
该层参数和：256
该层的结构：[1, 256, 1, 1]
该层参数和：256
该层的结构：[256, 256, 1, 1]
该层参数和：65536
该层的结构：[256]
该层参数和：256
该层的结构：[384, 256, 3, 3]
该层参数和：884736
该层的结构：[384]
该层参数和：384
该层的结构：[1, 384, 1, 1]
该层参数和：384
该层的结构：[1, 384, 1, 1]
该层参数和：384
该层的结构：[384, 384, 1, 1]
该层参数和：147456
该层的结构：[384]
该层参数和：384
该层的结构：[50, 384, 1, 1]
该层参数和：19200
该层的结构：[50]
该层参数和：50
该层的结构：[50, 7200]
该层参数和：360000
该层的结构：[50]
该层参数和：50
该层的结构：[1, 50]
该层参数和：50
该层的结构：[1, 50]
该层参数和：50
该层的结构：[2, 50]
该层参数和：100
该层的结构：[2]
该层参数和：2
总参数数量和：2243182


# noe_fcl_Net          三个（卷积+1*1卷积）+fl层

In [ ]:
import torch
from torchvision.models import AlexNet
from torchviz import make_dot
import torch.nn as nn
CHANNELS = [20, 50, 500]
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torchdata

input=torch.rand(64,12,60,60)



class one_fl_Net(nn.Module):
    def __init__(self, input_channels):
        super(one_fl_Net, self).__init__()
        
#         self.conv1 = nn.Conv2d(input_channels, CHANNELS[0], 5)
#         self.pool = nn.MaxPool2d(2, 2)
        
        self.a_1 = nn.Conv2d(12, 96, kernel_size=11, stride=4, padding=0)
        self.a_one = nn.Conv2d(96, 96, kernel_size=1)
        self.pool=nn.MaxPool2d(kernel_size=2, stride=1)
        
        self.b_1 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.b_one = nn.Conv2d(256, 256, kernel_size=1)
        self.pool=nn.MaxPool2d(kernel_size=2, stride=1)
        
        self.c_1 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.c_one = nn.Conv2d(384, 384, kernel_size=1)
        self.pool=nn.MaxPool2d(kernel_size=2, stride=1)
        
        self.consoft = nn.Conv2d(384,50,kernel_size=1,stride=1,padding=1)
        self.fc1 = nn.Linear(50 * 12 * 12, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
               
        x = F.relu(self.a_1(x))
        x = F.relu(self.a_one(F.relu(self.a_one(x))))
        x = self.pool(x)
        
        x = F.relu(self.b_1(x))
        x = F.relu(self.b_one(F.relu(self.b_one(x))))
        x = self.pool(x)
        
        x = F.relu(self.c_1(x))
        x = F.relu(self.c_one(F.relu(self.c_one(x))))              
        x = self.pool(x)
        
        x = self.consoft(x)   

        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model=one_one_Net(12)
y=model(input)
g=make_dot(y)
params = list(model.parameters())
k = 0
for i in params:
        l = 1
        print("该层的结构：" + str(list(i.size())))
        for j in i.size():
                l *= j
        print("该层参数和：" + str(l))
        k = k + l
print("总参数数量和：" + str(k))